동요학습 -> 작곡 모델

In [1]:
code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}
idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}
seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4',
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

g8, e8, e4, f8 -> 음표예측 d8  
e8, e4, f8, d8 -> 음표예측 d4  
g8, g8, e8, e9 -> 음표예측 e4 

In [2]:
def seq2dataset(seq, windoeSize):
    dataset=[]
    for i in range(len(seq)- windoeSize):
        subset = seq[i : (i+windoeSize+1)]
        dataset.append([code2idx[item] for item in subset])
    return np.array(dataset)

dataset=seq2dataset(seq, windoeSize=4) #윈도우 사이즈는 입력음표 개수

In [3]:
print(dataset)

[[11  9  2 10  8]
 [ 9  2 10  8  1]
 [ 2 10  8  1  7]
 [10  8  1  7  8]
 [ 8  1  7  8  9]
 [ 1  7  8  9 10]
 [ 7  8  9 10 11]
 [ 8  9 10 11 11]
 [ 9 10 11 11  4]
 [10 11 11  4 11]
 [11 11  4 11  9]
 [11  4 11  9  9]
 [ 4 11  9  9  9]
 [11  9  9  9 10]
 [ 9  9  9 10  8]
 [ 9  9 10  8  1]
 [ 9 10  8  1  7]
 [10  8  1  7  9]
 [ 8  1  7  9 11]
 [ 1  7  9 11 11]
 [ 7  9 11 11  9]
 [ 9 11 11  9  9]
 [11 11  9  9  2]
 [11  9  9  2  8]
 [ 9  9  2  8  8]
 [ 9  2  8  8  8]
 [ 2  8  8  8  8]
 [ 8  8  8  8  8]
 [ 8  8  8  8  9]
 [ 8  8  8  9  3]
 [ 8  8  9  3  9]
 [ 8  9  3  9  9]
 [ 9  3  9  9  9]
 [ 3  9  9  9  9]
 [ 9  9  9  9  9]
 [ 9  9  9  9 10]
 [ 9  9  9 10  4]
 [ 9  9 10  4 11]
 [ 9 10  4 11  9]
 [10  4 11  9  2]
 [ 4 11  9  2 10]
 [11  9  2 10  8]
 [ 9  2 10  8  1]
 [ 2 10  8  1  7]
 [10  8  1  7  9]
 [ 8  1  7  9 11]
 [ 1  7  9 11 11]
 [ 7  9 11 11  9]
 [ 9 11 11  9  9]
 [11 11  9  9  2]]


In [4]:
import numpy as np
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.datasets import reuters
from keras.utils import np_utils, to_categorical

In [10]:
dataset.shape #(50, 5)
xTrain = dataset[:, 0:4]
yTrain = dataset[:,4]

##LSTM
xTrain= np.reshape(xTrain, (50,4,1))

xTrain =xTrain/11
yTrain =np_utils.to_categorical(yTrain)

In [11]:
oneHotVecSize= yTrain.shape[1] #12

In [21]:
############바뀌는부분###############
#모델 구성, 학습, 평가, 사용
#단순 신경망 -> 정확도 92%
# model = Sequential()
# model.add(Dense(128, input_dim = 4, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(oneHotVecSize, activation='softmax')) 

In [16]:
########LSTM 구성##############
model = Sequential()
model.add(LSTM(128, batch_input_shape = (1, 4,1), stateful=True )) # 4:입력데이터 길이
model.add(Dense(oneHotVecSize, activation='softmax')) 

In [17]:
model.compile(loss ='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

for epoch_idx in range(2000):
    print('epochs : ' +str(epoch_idx))
    model.fit(xTrain, yTrain, epochs=1, batch_size=1, shuffle=False)
    model.reset_states()

epochs : 0
50/50 [==============================] - 0s 2ms/step - loss: 2.3551 - accuracy: 0.2400
epochs : 1
50/50 [==============================] - 0s 2ms/step - loss: 2.0159 - accuracy: 0.3400
epochs : 2
50/50 [==============================] - 0s 1ms/step - loss: 1.9586 - accuracy: 0.3400
epochs : 3
50/50 [==============================] - 0s 1ms/step - loss: 1.9424 - accuracy: 0.3400
epochs : 4
50/50 [==============================] - 0s 1ms/step - loss: 1.9326 - accuracy: 0.3400
epochs : 5
50/50 [==============================] - 0s 1ms/step - loss: 1.9255 - accuracy: 0.3400
epochs : 6
50/50 [==============================] - 0s 2ms/step - loss: 1.9199 - accuracy: 0.3600
epochs : 7
50/50 [==============================] - 0s 1ms/step - loss: 1.9152 - accuracy: 0.3600
epochs : 8
50/50 [==============================] - 0s 1ms/step - loss: 1.9110 - accuracy: 0.3600
epochs : 9
50/50 [==============================] - 0s 2ms/step - loss: 1.9071 - accuracy: 0.3600
epochs : 10
50/50 [=

In [14]:
model.evaluate(xTrain, yTrain)

2/2 [==============================] - 0s 998us/step - loss: 0.2555 - accuracy: 0.9000


[0.2555176615715027, 0.8999999761581421]

In [27]:
#예측
seqOut = ['g8', 'e8', 'e4', 'f8']
preOut =model.predict(xTrain)

In [31]:
for i in range(50):
    idx =np.argmax(preOut[i])
    seqOut.append(idx2code[idx])

print(seqOut)

['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'e8', 'f8', 'g8', 'g8', 'g4', 'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4', 'd8', 'd8', 'd8', 'd8', 'd8', 'd8', 'f4', 'e8', 'e8', 'e8', 'e8', 'f8', 'f8', 'd8', 'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4', 'd8', 'd4', 'c8', 'e8', 'e8', 'f8', 'g8', 'g8', 'g4', 'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4', 'd8', 'd8', 'd8', 'd8', 'd8', 'd8', 'f4', 'e8', 'e8', 'e8', 'e8', 'f8', 'f8', 'd8', 'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']


In [29]:
# seqOut에는 최종악보가 저장된다

In [35]:
# import wave
# !pip install pygame
from pygame import mixer

In [36]:
mixer.init()
mixer.music.load('sound1.mp3')
mixer.music.play()

error: DirectSoundCreate: No audio device found

In [37]:
#LSTM으로 재구성
